In [ ]:
import asyncio
import os
from dotenv import load_dotenv
from browser_use import Agent, ChatOpenAI, Browser, logger, BrowserSession
from browser_use import Tools, Agent, Controller, ActionResult
from browser_use.browser.events import UploadFileEvent
from browser_use.llm import ChatDeepSeek
import sys

tools = Tools()

@tools.action(description='Ask human for help with a question')
def ask_human(question: str) -> ActionResult:
    answer = input(f'{question} > ')
    return f'The human responded with: {answer}'


@tools.action('Save data to file')
def save_data(data: list, filename: str) -> str:
    with open(filename, 'w') as f:
        json.dump(data, f)
    return f'Saved {len(data)} items to {filename}'


browser = Browser(
    cdp_url= 'ws://localhost:9222/devtools/browser/7a9c70d2-aa33-43ad-8167-bb1485441668'
)

@tools.action('Fill common form fields')
async def fill_form_fields(browser_session: BrowserSession):
    """
    Automatically fill standard form fields like email, postal code, city, and address.
    Adjusts to whatever matching inputs exist on the page.
    """
    page = await browser_session.must_get_current_page()

    # Mapping of CSS selectors → example values
    field_values = {
        'input[name="email"]': "user@example.com",
        'input[name="postal"]': "70173",              
        'input[name="zipcode"]': "70173",
        'input[name="city"]': "Stuttgart",
        'input[name="address"]': "Königstraße 1, Stuttgart",
        'input[name="street"]': "Königstraße 1",
    }

    filled_fields = []

    for selector, value in field_values.items():
        elements = await page.get_elements_by_css_selector(selector)
        if elements:
            await elements[0].fill(value)
            filled_fields.append(f"{selector} → {value}")

    if filled_fields:
        return ActionResult(message=f"Filled fields: {', '.join(filled_fields)}")
    else:
        return ActionResult(message="No matching form fields found on this page.")

action_to_take = """
  Go to duckduck go and search for the weather
"""
async def main():
    # Initialize LLM
    llm = ChatDeepSeek(
        base_url='https://api.deepseek.com/v1',
        model='deepseek-chat',
        api_key="KEY-HERE",
    )


    # Create the agent
    agent = Agent(
        task=action_to_take,
        llm=llm,
        tools = tools, 
        browser = browser,
        file_system_path='/home/tom/WD',
        available_file_paths = "/home/tom/wallpapers/Marmotamaps_Wallpaper_PragserWildsee_Desktop_1920x1080.jpg",
        override_system_message=(
        "When you find an <input type='file'> element, do NOT click it directly. "
        "Instead, use the provided tool 'Upload file to interactive element with file path'. "
        "You must call that tool with the correct element index and file path."
    ),
    )
    # Run the agent
    await agent.run(max_steps=10)

if __name__ == "__main__":
    await main()


INFO     [Agent] 🔗 Found URL in task: https://de.indeed.com/?r=us&vjk=e3273c6b845351cd, adding as initial action...
WARNING  [Agent] ⚠️ DeepSeek models do not support use_vision=True yet. Setting use_vision=False for now...
INFO     [Agent] 🎯 Task: 
  Go to https://de.indeed.com/?r=us&vjk=e3273c6b845351cd and save the job info to job_info.txt

INFO     [Agent] 📦 Newer version available: 0.9.4 (current: 0.9.3). Upgrade with: uv add browser-use@0.9.4
INFO     [Agent] Starting a browser-use agent with version 0.9.3, with provider=deepseek and model=deepseek-chat
INFO     [Agent]   ▶️   navigate: url: https://de.indeed.com/?r=us&vjk=e3273c6b845351cd, new_tab: False
INFO     [tools] 🔗 Navigated to https://de.indeed.com/?r=us&vjk=e3273c6b845351cd
INFO     [Agent] 

INFO     [Agent] 📍 Step 1:
INFO     [Agent]   🧠 Memory: Already on the requested URL https://de.indeed.com/?r=us&vjk=e3273c6b845351cd, but page appears empty with preloader visible.
INFO     [Agent]   🎯 Next goal: Wait for page to